In [ ]:
import torch
import ultralytics
import cv2
import os
import json
from ultralytics import YOLO  
import val
from val import yolo2coco, coco_eval
from pathlib import Path



In [60]:

# Define your classes here (same order as during YOLO training)
classes = ["pedestrians", "bikes", "bicyclists", "e-scooters", "e-scooterists"]

name_model="yolo11x"
# Load YOLO model
model = YOLO("algos/"+name_model+".pt")

video_name='Test-360'
# Open the video file
video_path = "Videos/" + video_name + ".mp4"
cap = cv2.VideoCapture(video_path)

# Get video frame dimensions and fps
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Create output directories if they don't exist
images_dir = "yolo/datasets_"+ video_name +"/one/images"
labels_dir = "yolo/datasets_"+ video_name +"/one/labels"
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Create the classes.txt file
classes_file = "yolo/datasets_"+ video_name +"/one/classes.txt"
with open(classes_file, "w") as class_f:
    for cls in classes:
        class_f.write(f"{cls}\n")

print(f"Classes file created at: {classes_file}")

# Define the codec and create a VideoWriter object
output_path = "predictions/annotated_video_"+ video_name + '_'+name_model +".mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

frame_idx = 0  # Frame counter

# Process video frames
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Run YOLO detection
    results = model.track(frame, persist=True, imgsz=1280, conf=0.3, classes=[0, 1])

    # Visualize results on the frame
    annotated_frame = results[0].plot()

    # Save the current frame as an image for Label Studio
    image_file = os.path.join(images_dir, f"frame_{frame_idx:04d}.jpg")
    cv2.imwrite(image_file, frame)

    # Save detections in YOLO format
    label_file = os.path.join(labels_dir, f"frame_{frame_idx:04d}.txt")
    with open(label_file, "w") as f:
        for r in results[0].boxes:
            cls = int(r.cls)
            x_center, y_center, w, h = r.xywhn[0]  # normalized (x_center, y_center, width, height)

            # Write to YOLO label file
            f.write(f"{cls} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

          
      

    # Write the annotated frame to the output video
    out.write(annotated_frame)

    # Display the annotated frame (optional, can comment out)
    # cv2.imshow("Annotated Frame", annotated_frame)

    # Break loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    frame_idx += 1  # Increment frame counter


# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


Classes file created at: yolo/datasets_Test-360/one/classes.txt

0: 640x1280 13 persons, 1725.5ms
Speed: 4.6ms preprocess, 1725.5ms inference, 6.7ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 12 persons, 860.0ms
Speed: 4.5ms preprocess, 860.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 11 persons, 740.5ms
Speed: 3.7ms preprocess, 740.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 12 persons, 745.5ms
Speed: 3.1ms preprocess, 745.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 12 persons, 737.4ms
Speed: 3.3ms preprocess, 737.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 11 persons, 746.3ms
Speed: 3.4ms preprocess, 746.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x1280 12 persons, 752.0ms
Speed: 3.3ms preprocess, 752.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 1280)

0: 640x12

In [ ]:
import yaml

# Définition du dataset
dataset = {
    "train": "/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/train",
    "val": "/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/val",
    "nc": 5,  # Nombre de classes
    "names": ['pedestrians', 'bikes', 'bicyclists', 'e-scooterists', 'e-scooters'] # Classes
}

# Sauvegarde dans un fichier
with open("dataset.yaml", "w") as file:
    yaml.dump(dataset, file, default_flow_style=False)

print("Fichier dataset.yaml créé avec succès !")



## Train

In [198]:
results = model.train(data="dataset.yaml", epochs=100, imgsz=1280)

New https://pypi.org/project/ultralytics/8.3.94 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
engine/trainer: task=detect, mode=train, model=algos/MicroVision-Chalmers-x-20250304.pt, data=dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fram

train: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_GS010646/one/labels... 2029 images, 10 backgrounds, 0 corrupt: 100%|██████████| 2039/2039 [00:00<00:00, 2909.17it/s]


train: New cache created: /Users/martin.dejaeghere/Microsint2/yolo/datasets_GS010646/one/labels.cache


val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/val/labels.cache... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<?, ?it/s]

Plotting labels to runs/detect/train/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1280 train, 1280 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/128 [00:21<?, ?it/s]


KeyboardInterrupt: 

## Evaluation


In [75]:

# Validate with a custom dataset
name_model="MicroVision-Chalmers-x-distorted-20250318"
# Load YOLO model
model = YOLO("algos/"+name_model+".pt")

metrics = model.val(data="dataset.yaml",conf=0.4) 
print(metrics.box.map)  # map50-95

Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
YOLO11x summary (fused): 190 layers, 56,832,799 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/labels.cache... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:54<00:00, 38.18s/it]


                   all         46        563      0.908      0.636      0.794      0.665
                person         46        420       0.88       0.54      0.733      0.581
               bicycle         23         67      0.927      0.567       0.76      0.629
               cyclist         21         28      0.769      0.714      0.805       0.71
             e-scooter         20         39      0.964      0.692      0.841      0.717
          e-scooterist          8          9          1      0.667      0.833      0.689
Speed: 1.3ms preprocess, 2453.7ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val22
0.6653092581562342


In [76]:
name_model="MicroVision-Chalmers-x-only-distorted-20250320"
model=YOLO("algos/"+name_model+".pt")

# Validate with a custom dataset
metrics = model.val(data="dataset.yaml",conf=0.4)  
print(metrics.box.map)  


Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
YOLO11x summary (fused): 190 layers, 56,832,799 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/labels.cache... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:54<00:00, 38.11s/it]


                   all         46        563      0.855      0.432      0.668      0.505
                person         46        420      0.981      0.252      0.617      0.432
               bicycle         23         67      0.875      0.313      0.611      0.438
               cyclist         21         28       0.75      0.536      0.676      0.567
             e-scooter         20         39          1      0.615      0.808      0.581
          e-scooterist          8          9      0.667      0.444       0.63      0.506
Speed: 1.4ms preprocess, 2453.2ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/val23
0.5047384110658028


In [90]:
name_model="MicroVision-Chalmers-x-20250304"
model=YOLO("algos/"+name_model+".pt")

# Validate with a custom dataset
metrics = model.val(data="dataset.yaml",conf=0.4)  # output optional
print(metrics.box.map)  # map50-95


Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
YOLO11x summary (fused): 190 layers, 56,832,799 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/test/labels... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<00:00, 3116.73it/s]

val: New cache created: /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:52<00:00, 37.59s/it]


                   all         46        563      0.792      0.739      0.832      0.758
                person         46        420       0.95      0.583      0.777      0.706
               bicycle         23         67      0.939      0.687      0.829      0.731
               cyclist         21         28      0.714      0.714      0.804        0.8
             e-scooter         20         39      0.914      0.821      0.888      0.874
          e-scooterist          8          9      0.444      0.889      0.863       0.68
Speed: 2.4ms preprocess, 2417.6ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val31
0.758364834795386


In [42]:
# Définition du dataset
dataset = {
    "train": "/Users/martin.dejaeghere/Microsint2/yolo/datasets_GS010646",
    "val": "/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie",
    "nc": 2,  # Nombre de classes
    "names": ['persons', 'bikes'] # Classes
}

# Sauvegarde dans un fichier
with open("dataset.yaml", "w") as file:
    yaml.dump(dataset, file, default_flow_style=False)

print("Fichier dataset.yaml créé avec succès !")

name_model="yolo11x"
model=YOLO("algos/"+name_model+".pt")

# Validate with a custom dataset
metrics = model.val(data="dataset.yaml",conf=0.4,imgsz=1280,classes=[0, 1])  
print(metrics.box.map)  # map50-95


Fichier dataset.yaml créé avec succès !
Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
YOLO11x summary (fused): 190 layers, 56,919,424 parameters, 0 gradients, 194.9 GFLOPs


val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/labels.cache... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:56<00:00, 38.75s/it]


                   all         46        487      0.699      0.568        0.6      0.465
                person         46        420      0.772      0.614      0.657      0.492
               bicycle         23         67      0.625      0.522      0.544      0.438
Speed: 1.4ms preprocess, 2494.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val19
0.46490512603597517


In [196]:

model_path='/Users/martin.dejaeghere/Microsint2/algos/MicroVision-Chalmers-x-distorted-20250318.pt'
yaml_path='/Users/martin.dejaeghere/Microsint2/dataset.yaml'
val_path='/Users/martin.dejaeghere/Microsint2/runs/detect'
data_path='/Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one'

In [197]:
coco_eval(model_path, yaml_path, val_path, data_path, 128, 288, split="val", iou=0.45, agnostic_nms=True)

Loading model ...
Run validation ...
Ultralytics 8.3.87 🚀 Python-3.11.11 torch-2.6.0 CPU (Apple M3 Pro)
YOLO11x summary (fused): 190 layers, 56,832,799 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /Users/martin.dejaeghere/Microsint2/yolo/datasets_tolabel_copie/one/val/labels.cache... 46 images, 0 backgrounds, 0 corrupt: 100%|██████████| 46/46 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [01:56<00:00, 38.87s/it]


                   all         46        563      0.864      0.646      0.776      0.617
                person         46        420      0.877      0.578       0.73      0.528
               bicycle         23         67      0.906      0.582      0.772      0.585
               cyclist         21         28        0.7      0.679       0.76      0.663
             e-scooter         20         39      0.904      0.727      0.842      0.676
          e-scooterist          8          9      0.934      0.667      0.777      0.635
Speed: 1.4ms preprocess, 2502.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Saving runs/detect/detect/predictions.json...
Results saved to runs/detect/detect
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...


FileNotFoundError: [Errno 2] No such file or directory: '/Users/martin.dejaeghere/Microsint2/runs/detect/predictions.json'